First, bring in our libraries.

In [2]:
setwd("../src/R/")

In [3]:
source("./Sampler_SE.R")

In [4]:
ie_ref_data_SE <- read.csv("../../raw_data/ie.rnd20.SE.csv")
sample_years_SE <- read.csv("../../raw_data/year.se.csv")

In [5]:
library(foreach)
my.cluster <- parallel::makeCluster(24, type="FORK")  # adjust number of cores based on your system
doParallel::registerDoParallel(cl = my.cluster)

Now we run the simulator.

In [6]:
ct_p_values <- (0:10)/10

In [7]:
R0_values <- c(2, 3, 4, 5)
pop_sizes <- c(25, 30, 40, 50)

In [28]:
nsamples_per <- 10000  # Small batches to calibrate

state_counter <- "22XLaa"

r0_iter <- rep(R0_values, each = length(pop_sizes))
ps_iter <- rep(pop_sizes, times = length(R0_values))

metadata_table <- data.frame(ps=ps_iter, r0=r0_iter)
# This probably doesn't need to be a parquet, but we'll take it anyways
names(metadata_table) <- c("pop_size", "R0")
arrow::write_parquet(metadata_table, paste0("../../data/RealDataSplit/metadata.small", state_counter, ".rnd20.SE.simulated.parquet"))
print(paste0("../../data/RealDataSplit/metadata.small", state_counter, ".rnd20.SE.simulated.parquet"))


result <- foreach (ps = ps_iter, r0 = r0_iter) %dopar% {
    set.seed(1949)
    si_df <- data.frame()
    ei_df <- data.frame()
    trees_df <- data.frame()
    for (ct_p in ct_p_values) {
        si_values = c()
        ei_values = c()
        # trees <- list()
        for (i in 1:nsamples_per) {
            # print(i)
            # set.seed(i)
                        invisible(res <- simulate_SE_EU_HIV(sample_size = 20, ct_p = ct_p, R0=r0,
                                        sample_times_data = sample_years_SE$Freq, 
                                        sample_times_times = sample_years_SE$X, pop_size=ps,
                                        init_pop_size = 7,
                                        l1=12, l2=78, stretch=0, front_ratio = 3, mu_factor=1))
            si_values = c(si_values, res$sackin_index)
            if (res$sackin_index <= 88) {
                print(c(i, res$sackin_index))
            }
            ei_values = c(ei_values, res$EIr)
            # trees[[i]] <- ape::read.tree(text = res$newick_tree_string)
        }
        # si_df$ct_p <- si_values
        si_df <- rbind(si_df, data.frame(ct_p = rep(ct_p, nsamples_per), si = si_values))
        ei_df <- rbind(ei_df, data.frame(ct_p = rep(ct_p, nsamples_per), ei = ei_values))
        # trees_df <- rbind(trees_df, data.frame(ct_p = rep(ct_p, nsamples_per), tree = trees))
        print("X")
        print(c(min(si_values), mean(si_values), max(si_values), var(si_values)))
        flush.console()
    }
    arrow::write_parquet(si_df, paste0("../../data/RealDataSplit/sackin.small", state_counter, ".rnd20.SE.simulated.", ps, ".", r0, ".parquet"))
    arrow::write_parquet(ei_df, paste0("../../data/RealDataSplit/ei.small", state_counter, ".rnd20.SE.simulated.", ps, ".", r0, ".parquet"))
}


[1] "../../data/RealDataSplit/metadata.small22XLaa.rnd20.SE.simulated.parquet"


In [9]:
nsamples_per <- 10000  # Small batches to calibrate

state_counter <- "25XLaa"

r0_iter <- rep(R0_values, each = length(pop_sizes))
ps_iter <- rep(pop_sizes, times = length(R0_values))

metadata_table <- data.frame(ps=ps_iter, r0=r0_iter)
# This probably doesn't need to be a parquet, but we'll take it anyways
names(metadata_table) <- c("pop_size", "R0")
arrow::write_parquet(metadata_table, paste0("../../data/RealDataSplit/metadata.small", state_counter, ".rnd20.SE.simulated.parquet"))
print(paste0("../../data/RealDataSplit/metadata.small", state_counter, ".rnd20.SE.simulated.parquet"))


result <- foreach (ps = ps_iter, r0 = r0_iter) %dopar% {
    set.seed(1949)
    si_df <- data.frame()
    ei_df <- data.frame()
    trees_df <- data.frame()
    for (ct_p in ct_p_values) {
        si_values = c()
        ei_values = c()
        # trees <- list()
        for (i in 1:nsamples_per) {
            # print(i)
            # set.seed(i)
                        invisible(res <- simulate_SE_EU_HIV(sample_size = 20, ct_p = ct_p, R0=r0,
                                        sample_times_data = sample_years_SE$Freq, 
                                        sample_times_times = sample_years_SE$X, pop_size=ps,
                                        init_pop_size = 7, R0_init = 3,
                                        l1=12, l2=78, stretch=0, front_ratio = 3, mu_factor=1))
            si_values = c(si_values, res$sackin_index)
            if (res$sackin_index <= 88) {
                print(c(i, res$sackin_index))
            }
            ei_values = c(ei_values, res$EIr)
            # trees[[i]] <- ape::read.tree(text = res$newick_tree_string)
        }
        # si_df$ct_p <- si_values
        si_df <- rbind(si_df, data.frame(ct_p = rep(ct_p, nsamples_per), si = si_values))
        ei_df <- rbind(ei_df, data.frame(ct_p = rep(ct_p, nsamples_per), ei = ei_values))
        # trees_df <- rbind(trees_df, data.frame(ct_p = rep(ct_p, nsamples_per), tree = trees))
        print("X")
        print(c(min(si_values), mean(si_values), max(si_values), var(si_values)))
        flush.console()
    }
    arrow::write_parquet(si_df, paste0("../../data/RealDataSplit/sackin.small", state_counter, ".rnd20.SE.simulated.", ps, ".", r0, ".parquet"))
    arrow::write_parquet(ei_df, paste0("../../data/RealDataSplit/ei.small", state_counter, ".rnd20.SE.simulated.", ps, ".", r0, ".parquet"))
}


[1] "../../data/RealDataSplit/metadata.small25XLaa.rnd20.SE.simulated.parquet"
